In [1]:
import pandas as pd
import json

In [2]:
# load sets to list from json file
file_name = 'scraped_BE_final.json'

with open(file_name, 'r') as file:
    json_data = json.load(file)

index_ls = []
set_ls = []
for json_set in json_data:
    name = json_set.keys()
    set_info = json_set.values()

    index_ls.append(*name)
    set_ls.append(*set_info)

In [3]:
# transform nested dictionaries to flatten dictionary 
def flatten_dict_set_info(dictionary: dict) -> dict:
    temp_dict = {}

    theme = dictionary['set_info'].get('theme', None)
    year = dictionary['set_info'].get('year', None)
    av = dictionary['set_info'].get('availability', None)
    pieces = dictionary['set_info'].get('Pieces', None)
    minifigs = dictionary['set_info'].get('Minifigs', None)

    temp_dict['Theme'] = theme
    temp_dict['Year'] = year
    temp_dict['Availability'] = av
    temp_dict['Pieces'] = pieces
    temp_dict['Minifigs'] = minifigs

    return temp_dict

def flatten_dict_prices(dictionary: dict) -> dict:
    temp_dict = {}

    retail = dictionary['prices'].get('Retail', None)
    value = dictionary['prices'].get('Value', None)

    temp_dict['Retail'] = retail
    temp_dict['Value'] = value

    return temp_dict

def convert_to_dict(stores: list) -> dict:
    temp_dict = {'LEGO': 0, 'Amazon': 0, 'Bricklink': 0, 'StockX': 0, 'eBay': 0}

    for store in stores:
        match store:
            case 'LEGO':
                temp_dict['LEGO'] = 1
            case 'StockX':
                temp_dict['StockX'] = 1
            case 'Amazon':
                temp_dict['Amazon'] = 1
            case 'Bricklink':
                temp_dict['Bricklink'] = 1
            case _:
                temp_dict['eBay'] = 1 
    return temp_dict

list_to_df = []
error_ls = [] # for potential missing informations -> finally only set_info
for num, lego_set in enumerate(set_ls):
    set_dict = {}

    try:
        temp = flatten_dict_set_info(lego_set)
        set_dict.update(temp)
    except:
        print(f'An error occurred for set_info - iteration number: {num}')
        error_ls.append(num)

    try:
        temp = flatten_dict_prices(lego_set)
        set_dict.update(temp)
    except:
        print(f'An error occurred for prices - iteration number: {num}')
        error_ls.append(num)
    
    try:
        temp = convert_to_dict(lego_set['stores'])
        set_dict.update(temp)
    except:
        print(f'An error occurred for stires - iteration number: {num}')
        error_ls.append(num)

    list_to_df.append(set_dict)

An error occurred for set_info - iteration number: 728


In [4]:
# adding manually missing values
print(error_ls)
print(list_to_df[error_ls[0]])

errored_set = list_to_df[error_ls[0]]
temp = {'Theme': 'Value Packs', 'Year': 2018, 'Availability': 'Retired', 'Pieces': 0, 'Minifigs': 0}
errored_set.update(temp)

list_to_df[error_ls[0]] = errored_set

# I'll remove Value Packs anyway...

[728]
{'Retail': 339.99, 'Value': 489.05, 'LEGO': 0, 'Amazon': 0, 'Bricklink': 1, 'StockX': 0, 'eBay': 0}


In [39]:
# convert list of dicts to dataframe

set_df = pd.DataFrame(list_to_df, index=index_ls)
set_df # check missing values using Data Wrangler

# missing pieces -> promotional minifig -> can be set to zero
# missinc minifigs -> should be set to zero

# Change Availability column to numeric
qry = (set_df['Availability'] == 'Exclusive')
set_df['Exclusive'] = 0
set_df.loc[qry, 'Exclusive'] = 1

# Fill missing pieces
set_df.loc[set_df['Pieces'].isna(), 'Pieces'] = 0

# Fill missing minifigs
set_df.loc[set_df['Minifigs'].isna(), 'Minifigs'] = 0

# Fill promotional sets with retail zero and create promotional column
qry = (set_df['Retail'].isin(['Promotional', 'Promotional or Unknown']))
set_df['Promotional'] = 0
set_df.loc[qry, 'Promotional'] = 1
set_df.loc[qry, 'Retail'] = 0.00

# Fill value of sets on market
qry = (set_df['Value'].isin(['Not yet released', 'Available at retail']))
set_df.loc[qry, 'Value'] = set_df.loc[qry, 'Retail']

# Transform Year Column
set_df['Years'] = 2025 - set_df['Year']

# Drop columns and change datatype
set_df.drop(columns=['Theme', 'Year', 'Availability'], inplace=True)
set_df['Minifigs'] = set_df['Minifigs'].astype(int)
set_df['Retail'] = set_df['Retail'].astype(float)
set_df['Value'] = set_df['Value'].astype(float)

In [ ]:
# Check head
set_df.head(n= 15)

,Pieces,Minifigs,Retail,Value,LEGO,Amazon,Bricklink,StockX,eBay,Exclusive,Promotional,Years
75385 Battle on Peridea,382.0,5,54.99,54.99,1,1,0,0,1,0,0,1
75357 Ghost & Phantom II,1394.0,4,159.99,159.99,1,1,0,1,1,0,0,2
75364 New Republic E-wing vs. Shin Hati's Starfighter,1056.0,5,109.99,88.47,0,1,0,1,1,0,0,2
75362 Ahsoka Tano's T-6 Jedi Shuttle,599.0,4,79.99,79.99,1,1,0,1,1,0,0,2
75338 Ambush on Ferrix,679.0,3,69.99,76.01,0,1,1,1,1,0,0,3
75226 Inferno Squad Battle Pack,118.0,4,14.99,59.88,0,1,1,0,1,0,0,6
75134 Galactic Empire Battle Pack,109.0,4,12.99,52.97,0,1,1,1,1,0,0,9
75133 Rebel Alliance Battle Pack,101.0,4,12.99,39.41,0,1,1,0,1,0,0,9
11912 Star Wars Build Your Own Adventure Parts,69.0,1,11.99,19.99,0,1,0,0,1,0,0,9
75253 Droid Commander,1177.0,0,199.99,207.19,0,1,0,1,1,0,0,6


In [ ]:
# Check dtypes
set_df.dtypes

Pieces         float64
Minifigs         int64
Retail         float64
Value          float64
LEGO             int64
Amazon           int64
Bricklink        int64
StockX           int64
eBay             int64
Exclusive        int64
Promotional      int64
Years            int64
dtype: object

In [45]:
# write to parquet
file_out_name = 'clean_BE_df.parquet'

set_df.to_parquet(file_out_name)